# Classifying news articles with Naive Bayes

<a href="https://colab.research.google.com/github/chu-ise/413A-2022/blob/main/notebooks/07/07-1_news_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Once text data has been converted into numerical features, text classification works just like any other classification task.

## Imports

In [ ]:
%matplotlib inline

from pathlib import Path

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

## News article classification

We start with an illustration of the Naive Bayes model for news article classification using the BBC articles that we read as before to obtain a DataFrame with 2,225 articles from 5 categories.

### Read BBC articles

In [ ]:
import gdown
id = "15jqsbZeUVz09McM3_CMxmPK39TyBYKFD"
data_file = "bbc.zip"
gdown.cached_download(id=id, path=data_file, postprocess=gdown.extractall)

In [ ]:
path = Path('bbc')
files = sorted(list(path.glob('**/*.txt')))
doc_list = []
for i, file in enumerate(files):
    topic = file.parts[-2]
    article = file.read_text(encoding='latin1').split('\n')
    heading = article[0].strip()
    body = ' '.join([l.strip() for l in article[1:]])
    doc_list.append([topic, heading, body])

In [ ]:
docs = pd.DataFrame(doc_list, columns=['topic', 'heading', 'body'])
docs.info()

In [ ]:
docs.head()

### Create stratified train-test split

We split the data into the default 75:25 train-test sets, ensuring that the test set classes closely mirror the train set:

In [ ]:
y = pd.factorize(docs.topic)[0]
X = docs.body
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

### Vectorize text data

We proceed to learn the vocabulary from the training set and transforming both dataset using the CountVectorizer with default settings to obtain almost 26,000 features:

In [ ]:
vectorizer = CountVectorizer()
X_train_dtm = vectorizer.fit_transform(X_train)
X_test_dtm = vectorizer.transform(X_test)

In [ ]:
X_train_dtm.shape, X_test_dtm.shape

### Train Multi-class Naive Bayes model

In [ ]:
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

### Evaluate Results

We evaluate the multiclass predictions using accuracy to find the default classifier achieved almost 98%:

#### Accuracy

In [ ]:
accuracy_score(y_test, y_pred_class)

#### Confusion matrix

In [ ]:
pd.DataFrame(confusion_matrix(y_true=y_test, y_pred=y_pred_class))

### Vectorize text data as TFIDF

In [ ]:
vectorizer = TfidfVectorizer()
X_train_dtm = vectorizer.fit_transform(X_train)
X_test_dtm = vectorizer.transform(X_test)

nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

accuracy_score(y_test, y_pred_class)

In [ ]:
pd.DataFrame(confusion_matrix(y_true=y_test, y_pred=y_pred_class))